# Welcome to DFL-Colab!

This is an adapted version of the DFL for Google Colab.
Works only with DFL 1.0

# Overview

- Extractor works in full functionality.
- Training can work without preview.
- Converter works in full functionality.
- You can import/export workspace with your Google Drive.
- Import/export and another manipulations with workspace you can do in "Manage workspace" block
- Google Colab machine active for 12 hours. DFL-Colab makes a backup of your workspace in training mode, after 11 hours from the start of the session.
- Google does not like long-term heavy calculations. Therefore, for training more than two sessions in a row, use two Google accounts. It is recommended to split your training over 2 accounts, but you can use one Google Drive account to store your workspace.


## Prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.


In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")


# Check GPU

- Google Colab can provide you with one of Tesla graphics cards: K80, T4, P4 or P100
- Here you can check the model of GPU before using DeepFaceLab


In [ ]:
!nvidia-smi


# Clone Github repository and install requirements

- Clone Github repository or pull updates
- Requirements install is automatically


In [ ]:
# @title Install or update DeepFaceLab from Github

from google.colab import output
import IPython
import os
from pathlib import Path

Mode = "install"  # @param ["install", "update"]
Download_FFHQ = True  # @param {type:"boolean"}

base_dir = '/content/drive/MyDrive/DeepFaceLabProject'
dfl_dir = f"{base_dir}/DeepFaceLab"
pretrain_dir = f"{base_dir}/pretrain"
pretrain_faceset_zip = f"{base_dir}/pretrain_faceset.zip"
pretrain_Q96_dir = f"{base_dir}/pretrain_Q96"
pretrain_Q96_zip = f"{base_dir}/pretrain_Q96.zip"

workspace_dir = f"{base_dir}/workspace"
data_src_dir = f"{workspace_dir}/data_src"
aligned_data_src_dir = f"{data_src_dir}/aligned"
data_dst_dir = f"{workspace_dir}/data_dst"
aligned_data_dst_dir = f"{data_dst_dir}/aligned"
model_dir = f"{workspace_dir}/model"

pretrain_link = "https://github.com/chervonij/DFL-Colab/releases/download/"
pretrain_link = f"{pretrain_link}pretrain_GenericFFHQ/pretrain_FFHQ.zip" if Download_FFHQ else f"{pretrain_link}pretrain-CelebA/pretrain_CelebA.zip"

os.chdir(base_dir)
print("Current working directory:", os.getcwd())

if (Mode == "install"):
    !git clone "https://github.com/iperov/DeepFaceLab.git"
    os.chdir(dfl_dir)
    os.chdir(base_dir)
else:
    os.chdir(dfl_dir)
    !git pull

!pip install tqdm numpy==1.19.3 numexpr h5py==2.10.0 opencv-python ffmpeg-python==0.1.17 scikit-image==0.14.2 scipy==1.4.1 colorama tensorflow-gpu==2.4.0 tf2onnx==1.9.3
!pip install --upgrade scikit-image
!pip install colorama

if not Path(pretrain_dir).exists():
  print("Downloading Pretrain faceset ... ")
  !wget -q --no-check-certificate -r $pretrain_link -O $pretrain_faceset_zip
  !mkdir $pretrain_dir
  !unzip -q $pretrain_faceset_zip -d $pretrain_dir
  !rm $pretrain_faceset_zip

if not Path(pretrain_Q96_dir).exists():
  print("Downloading Q96 pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O $pretrain_Q96_zip
  !mkdir $pretrain_Q96_dir
  !unzip -q $pretrain_Q96_zip -d $pretrain_Q96_dir
  !rm $pretrain_Q96_zip


if not Path(workspace_dir).exists():
    !mkdir $workspace_dir
    !mkdir $data_src_dir
    !mkdir $aligned_data_src_dir
    !mkdir $data_dst_dir
    !mkdir $aligned_data_dst_dir


display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")


# Extract, sorting and faceset tools

- Extract frames for SRC or DST video.
- Denoise SRC or DST video. "Factor" param set intesity of denoising
- Detect and align faces with one of detectors. (S3FD - recommended). If you need, you can get frames with debug landmarks.
- Options "S3FD (Mark only)" and "S3FD (Extract unaligned)" are needed for the AVATAR model. Before using, read the guide in the DeepFaceLab repository.
- Export workspace to Google Drive after extract and sort it manually (In "Manage Workspace" block)
- You can enhance your facesets with DFL FacesetEnhancer. Enhanced images will replace yours in "aligned" directory.


In [ ]:
# @title Extract frames
Video = "data_src"  # @param ["data_src", "data_dst"]

base_dir = '/content/drive/MyDrive/DeepFaceLabProject'

%cd $base_dir

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
    cmd += " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
    cmd += " --input-file workspace/data_src.* --output-dir workspace/data_src/"

!python $cmd


In [ ]:
# @title Denoise frames
Data = "data_src"  # @param ["data_src", "data_dst"]
Factor = 1  # @param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/" + \
    Data+" --factor "+str(Factor)

base_dir = '/content/drive/MyDrive/DeepFaceLabProject'

%cd $base_dir
!python $cmd


In [ ]:
# @title Detect faces
Data = "data_src"  # @param ["data_src", "data_dst"]
# @param ["S3FD", "S3FD (Mark only)", "S3FD (Extract unaligned)", "MT"]
Detector = "S3FD"
Debug = False  # @param {type:"boolean"}

detect_type = "s3fd"
facetype = None

if Detector == "S3FD":
    detect_type = "s3fd"
elif Detector == "S3FD (Mark only)":
    detect_type = "s3fd"
    facetype = "mark_only"
elif Detector == "S3FD (Extract unaligned)":
    detect_type = "s3fd"
    facetype = "full_face_no_align"
elif Detector == "MT":
    detect_type = "mt"

folder = "workspace/"+Data
folder_align = folder+"/aligned"
debug_folder = folder_align+"/debug"

cmd = "DeepFaceLab/main.py extract --input-dir " + \
    folder+" --output-dir "+folder_align

if Debug:
    cmd += " --debug-dir "+debug_folder

cmd += " --detector "+detect_type

if facetype:
    cmd += " --face-type "+facetype

base_dir = '/content/drive/MyDrive/DeepFaceLabProject'

%cd $base_dir
!python $cmd


In [ ]:
# @title Sort aligned
Data = "data_src"  # @param ["data_src", "data_dst"]
# @param ["absdiff","hist", "hist-dissim", "face-yaw", "face-pitch", "vggface", "blur", "final", "final-no-blur"]
sort_type = "absdiff"

cmd = "DeepFaceLab/main.py sort --input-dir workspace/" + \
    Data+"/aligned --by "+sort_type

base_dir = '/content/drive/MyDrive/DeepFaceLabProject'

%cd $base_dir
!python $cmd


In [ ]:
# @title Faceset Enhancer
Data = "data_src"  # @param ["data_src", "data_dst"]

data_path = "/content/drive/MyDrive/DeepFaceLabProject/workspace/"+Data+"/aligned"

cmd = "/content/drive/MyDrive/DeepFaceLabProject/DeepFaceLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd


# Train model

- Choose your model type, but SAEHD is recommend for everyone
- Set model options on output field
- You can see preview manually, if go to model folder in filemanager and double click on preview.jpg file
- Your workspace will be archived and upload to mounted Drive after 11 hours from start session
- If you select "Backup_every_hour" option, your workspace will be backed up every hour.
- Also, you can export your workspace manually in "Manage workspace" block


In [ ]:
# ml_env = "/content/drive/MyDrive/ml_env/bin/activate"
ROOT_DIR = "/content/drive/MyDrive/DeepFaceLabProject/DeepFaceLab"
DFL_WORKSPACE = "/content/drive/MyDrive/DeepFaceLabProject/workspace"

# %cd $ROOT_DIR; pwd
!python $ROOT_DIR/main.py train \
    --training-data-src-dir "$DFL_WORKSPACE/data_src/aligned" \
    --training-data-dst-dir "$DFL_WORKSPACE/data_dst/aligned" \
    --pretraining-data-dir "$ROOT_DIR/pretrain" \
    --pretrained-model-dir "$ROOT_DIR/pretrain_Quick96" \
    --model-dir "$DFL_WORKSPACE/model" \
    --model Quick96

# Convert frames


In [ ]:
ROOT_DIR = "/content/drive/MyDrive/DeepFaceLabProject/DeepFaceLab"
DFL_WORKSPACE = "/content/drive/MyDrive/DeepFaceLabProject/workspace"

!python $ROOT_DIR/main.py videoed video-from-sequence \
    --input-dir "$DFL_WORKSPACE/data_dst/merged" \
    --output-file "$DFL_WORKSPACE/result.mp4" \
    --reference-file "$DFL_WORKSPACE/data_dst.*" \
    --include-audio

!python $ROOT_DIR/main.py videoed video-from-sequence \
    --input-dir "$DFL_WORKSPACE/data_dst/merged_mask" \
    --output-file "$DFL_WORKSPACE/result_mask.mp4" \
    --reference-file "$DFL_WORKSPACE/data_dst.*" \
    --lossless